Notebook for Extracting & Saving Raw Data

First:
- Loads daily stock prices using the SimFin API.
- Saves data to stock_prices.csv.
- Then reads it again and does basic inspection (head(), describe(), isna().mean()).

Then:
- Downloads annual income statements (load_income()).
- Saves to us_income_statements.csv.
- Downloads metadata about companies into us_companies_list.csv.

In [1]:
import os
from dotenv import load_dotenv
import requests
import json
import pandas as pd
import simfin as sf
from simfin.names import CLOSE, OPEN, HIGH, LOW, VOLUME, TICKER
from simfin.names import *

In [2]:
# load environment variables from the .env file
load_dotenv()

# get the API token from the environment variables
api_key = os.getenv('API_KEY')

In [3]:
os.makedirs("data/RAW", exist_ok=True)

In [4]:
def load_data():
    """
    Loads daily stock prices for all US companies from SimFin 
    and saves the raw data to a specified directory for processing.
    """

    # Set SimFin API key (Replace with your actual key)
    sf.set_api_key(api_key)

    # Set SimFin data directory (ensure it exists)
    sf.set_data_dir("~/simfin_data/")

    # Load daily stock prices for all US companies
    print("📥 Downloading US stock market data...")
    df_prices = sf.load_shareprices(market="us", variant="daily")

    # Reset index to make 'Date' a normal column
    df_prices = df_prices.reset_index()

    # Define save path - ADAPT TO YOUR LOCAL ENVIRONMENT
    save_path = "data/RAW/stock_prices.csv"

    # Save raw data to CSV
    df_prices.to_csv(save_path, index=False)

    print(f"✅ Data saved to: {save_path}")

# Run the function
load_data()

📥 Downloading US stock market data...
Dataset "us-shareprices-daily" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

c:\Users\SABIO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
✅ Data saved to: data/RAW/stock_prices.csv


In [5]:
# Read the saved CSV file into a DataFrame
df_prices = pd.read_csv("data/RAW/stock_prices.csv")

# Display the first few rows to confirm it loaded correctly
df_prices.head()

,Ticker,Date,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
0,A,2019-04-26,45846,76.98,77.46,76.30,77.42,74.36,1608922,NaN,317515869.0
1,A,2019-04-29,45846,77.47,78.44,77.32,77.33,74.27,2177700,NaN,317515869.0
2,A,2019-04-30,45846,77.44,78.59,77.38,78.50,75.39,1726239,NaN,317000000.0
3,A,2019-05-01,45846,78.49,78.92,77.28,77.47,74.40,1078572,NaN,317000000.0
4,A,2019-05-02,45846,77.41,78.34,77.15,78.20,75.10,946764,NaN,317000000.0


In [6]:
df_prices.describe()

,SimFinId,Open,High,Low,Close,Adj. Close,Volume,Dividend,Shares Outstanding
count,5.871346e+06,5.871346e+06,5.871346e+06,5.871346e+06,5.871346e+06,5.871346e+06,5.871346e+06,35886.000000,5.327273e+06
mean,5.949627e+06,1.615800e+04,1.625647e+04,1.602393e+04,1.611522e+04,1.611306e+04,1.783667e+06,0.455062,5.988376e+11
std,6.099718e+06,1.220804e+06,1.223978e+06,1.216359e+06,1.219076e+06,1.219076e+06,2.872576e+07,7.994542,6.171890e+13
min,1.800000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,6.277740e+05,7.800000e+00,8.010000e+00,7.570000e+00,7.800000e+00,7.320000e+00,3.597800e+04,0.120000,1.810615e+07
50%,1.326823e+06,2.025000e+01,2.070000e+01,1.984000e+01,2.025000e+01,1.883000e+01,2.435880e+05,0.250000,4.904499e+07
75%,1.103598e+07,5.172000e+01,5.265000e+01,5.075000e+01,5.169000e+01,4.874000e+01,9.385778e+05,0.450000,1.313580e+08
max,1.858941e+07,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.848998e+10,1500.000000,6.667887e+15


In [7]:
df_prices.isna().mean() # 99% of the dividend column is null for all tickers 

Ticker                0.000000
Date                  0.000000
SimFinId              0.000000
Open                  0.000000
High                  0.000000
Low                   0.000000
Close                 0.000000
Adj. Close            0.000000
Volume                0.000000
Dividend              0.993888
Shares Outstanding    0.092666
dtype: float64

In [8]:
# Download all US Income Statements
df_income = sf.load_income(variant='annual', market='us')

# Save to CSV
df_income.to_csv("data/RAW/us_income_statements.csv", index=False)

# to check, print all Revenue and Net Income for Microsoft (ticker MSFT).
print(df_income.loc['MSFT', [REVENUE, NET_INCOME]])

Dataset "us-income-annual" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


c:\Users\SABIO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


                  Revenue   Net Income
Report Date                           
2019-06-30   1.258430e+11  39240000000
2020-06-30   1.430150e+11  44281000000
2021-06-30   1.680880e+11  61271000000
2022-06-30   1.982700e+11  72738000000
2023-06-30   2.119150e+11  72361000000


In [9]:
df_income.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17555 entries, ('A', Timestamp('2019-10-31 00:00:00')) to ('ZYXI', Timestamp('2023-12-31 00:00:00'))
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   SimFinId                                  17555 non-null  int64         
 1   Currency                                  17555 non-null  object        
 2   Fiscal Year                               17555 non-null  int64         
 3   Fiscal Period                             17555 non-null  object        
 4   Publish Date                              17555 non-null  datetime64[ns]
 5   Restated Date                             17555 non-null  datetime64[ns]
 6   Shares (Basic)                            17403 non-null  float64       
 7   Shares (Diluted)                          17276 non-null  float64       
 8   Revenue                        

In [10]:
# Download the list of all US companies with metadata
df_companies = sf.load_companies(market='us')

# Save to CSV
df_companies.to_csv("data/RAW/us_companies_list.csv", index=False)

Dataset "us-companies" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... Done!


c:\Users\SABIO\Documents\GitHub\python-group-project-2\.venv\Lib\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,
